In [3]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# 1. Đọc Excel
df = pd.read_excel('data_cleaned.xlsx', sheet_name='ticket')
df_customer = pd.read_excel('data_cleaned.xlsx', sheet_name="customer")
df_film = pd.read_excel('data_cleaned.xlsx', sheet_name="film")
df_readme = pd.read_excel('data_cleaned.xlsx', sheet_name="readme")

# 3. Chuẩn bị connection string Windows Auth
conn_str = (
    r"Driver={ODBC Driver 17 for SQL Server};"
    r"Server=DESKTOP-DEM3AI5;"
    r"Database=data_cinema;"           
    r"Trusted_Connection=yes;"
)
# quote_plus để escape các ký tự đặc biệt
quoted = urllib.parse.quote_plus(conn_str)

# 4. Tạo engine SQLAlchemy
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted}")

# 5. Ghi DataFrame vào table dbo.ticket
df.to_sql(
    name="ticket",
    con=engine,
    schema="dbo",        
    if_exists="replace", 
    index=False
)
df_customer.to_sql('customer', con=engine, schema='dbo', if_exists='replace', index=False)
df_film.to_sql('film', con=engine, schema='dbo', if_exists='replace', index=False)
df_readme.to_sql('readme', con=engine, schema='dbo', if_exists='replace', index=False)



18

In [ ]:
import pandas as pd
import pyodbc

# Thiết lập kết nối tới SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-DEM3AI5;DATABASE=data_cinema;Trusted_Connection=yes')
# Viết truy vấn SQL
sql_query = """
WITH RECENCY AS (
    SELECT 
        customerid, 
        DATEDIFF(DAY, MAX(saledate), '2019-06-15') AS Recency -- Tính số ngày từ lần giao dịch gần nhất đến ngày hiện tại
    FROM dbo.ticket
    GROUP BY customerid
),
FREQUENCY AS (
    SELECT 
        customerid, 
        COUNT(*) AS Frequency -- Đếm số lần giao dịch của khách hàng
    FROM dbo.ticket
    GROUP BY customerid
),
MONETARY AS (
    SELECT 
        customerid, 
        SUM(total) AS Monetary -- Tổng chi tiêu của khách hàng
    FROM dbo.ticket
    GROUP BY customerid
)

-- Kết hợp các bảng Recency, Frequency và Monetary vào một bảng
SELECT R.CUSTOMERID, R.RECENCY, F.FREQUENCY, M.MONETARY,
    CASE 
        WHEN R.RECENCY <= 20 AND F.FREQUENCY >= 8 AND M.MONETARY > 800000 THEN 'VIP'
        WHEN R.RECENCY <= 30 AND F.FREQUENCY BETWEEN 5 AND 7 AND M.MONETARY BETWEEN 500000 AND 800000 THEN N'Khách hàng tiềm năng cao'
        WHEN R.RECENCY <= 90 AND F.FREQUENCY BETWEEN 2 AND 4 AND M.MONETARY BETWEEN 200000 AND 499999 THEN N'Khách hàng tiềm năng'
        ELSE N'Khách hàng cần chú ý'
    END AS CUSTOMER
FROM RECENCY R
JOIN FREQUENCY F ON R.CUSTOMERID = F.CUSTOMERID
JOIN MONETARY M ON R.CUSTOMERID = M.CUSTOMERID
ORDER BY CUSTOMER DESC
"""

# Đọc dữ liệu vào DataFrame
df = pd.read_sql(sql_query, conn)

# Đọc file Excel hiện tại (data_clean.xlsx)
excel_file = 'data_clean.xlsx'
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer: 
    df.to_excel(writer, sheet_name='Phân loại khách hàng', index=False)

# Đóng kết nối
conn.close()


C:\Users\admin\AppData\Local\Temp\ipykernel_24760\3363542322.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)
